# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
! pip install -q "datasets==2.15.0"
! pip install evaluate

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 785.9 kB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [11]:
from datasets import load_dataset
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig,AutoPeftModelForSequenceClassification
import torch
import pandas as pd

In [3]:


# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]


# View the dataset characteristics
dataset["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [4]:


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

In [5]:


model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

for param in model.parameters():
    param.requires_grad=False

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
tokenized_dataset["train"]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

In [7]:
tokenized_dataset["test"]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 1115
})

In [8]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)
model.eval()
trainer.predict(tokenized_dataset["test"])

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


PredictionOutput(predictions=array([[-0.22193393, -0.10221395],
       [-0.1512852 , -0.04220848],
       [-0.17859304, -0.08620155],
       ...,
       [-0.22084889, -0.08836212],
       [-0.20706062, -0.03743909],
       [-0.19740653, -0.04701708]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'test_loss': 0.7528449892997742, 'test_accuracy': 0.12914798206278028, 'test_runtime': 526.5337, 'test_samples_per_second': 2.118, 'test_steps_per_second': 0.266})

In [12]:
manual_review = tokenized_dataset["test"].select(
    [0, 1, 10, 22, 31, 115, 223, 292, 336, 442, 487, 558,669,771,884,993]
)

results = trainer.predict(manual_review)
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
df

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa...,1,0
1,Happy new years melody!\n,1,0
2,He is world famamus....\n,1,0
3,PRIVATE! Your 2003 Account Statement for shows...,1,1
4,URGENT! We are trying to contact U. Todays dra...,1,1
5,Networking technical support associate.\n,1,0
6,Im done. Just studyn in library\n,1,0
7,"SMS. ac JSco: Energy is high, but u may not kn...",1,1
8,Boy you best get yo ass out here quick\n,1,0
9,No he didn't. Spring is coming early yay!\n,1,0


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [13]:
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig

In [14]:
config = LoraConfig(task_type="SEQ_CLS",r=4,lora_alpha=32,lora_dropout=0.01,
                    target_modules=["q_lin","v_lin",])

lora_peft_model = get_peft_model(model, config)
lora_peft_model.print_trainable_parameters()

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

training_args = TrainingArguments(
    output_dir= "./data/spam_not_spam",
    learning_rate=1e-3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
ft_trainer = Trainer(
    model=lora_peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# train model
ft_trainer.train()
ft_trainer.evaluate()

trainable params: 1,257,988 || all params: 67,620,868 || trainable%: 1.8603547058875376


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.047996,0.987444


Checkpoint destination directory ./data/spam_not_spam/checkpoint-279 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.047995999455451965,
 'eval_accuracy': 0.9874439461883409,
 'eval_runtime': 218.8474,
 'eval_samples_per_second': 5.095,
 'eval_steps_per_second': 0.32,
 'epoch': 1.0}

In [15]:
lora_peft_model.save_pretrained("mani-fine-tuned", save_adapter=True, save_config=True)
lora_peft_model.merge_and_unload()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [16]:
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("mani-fine-tuned",  num_labels=2)
lora_model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.01, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=4, bias=Fal

In [17]:
manual_review = tokenized_dataset["test"].select(
    [0, 1, 10, 22, 31, 115, 223, 292, 336, 442, 487, 558,669,771,884,993]
)

In [18]:
predictions = []
for i in manual_review:
    input_tokens = tokenizer(i['sms'], return_tensors="pt")
    with torch.no_grad():
        logits = lora_model(**input_tokens).logits
        predicted_class_id = logits.argmax().item()
        predictions.append(predicted_class_id)

In [20]:
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in manual_review],
        "predictions": predictions,
        "labels": [item["label"] for item in manual_review],
    }
)
df

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa...,0,0
1,Happy new years melody!\n,0,0
2,He is world famamus....\n,0,0
3,PRIVATE! Your 2003 Account Statement for shows...,1,1
4,URGENT! We are trying to contact U. Todays dra...,1,1
5,Networking technical support associate.\n,0,0
6,Im done. Just studyn in library\n,0,0
7,"SMS. ac JSco: Energy is high, but u may not kn...",1,1
8,Boy you best get yo ass out here quick\n,0,0
9,No he didn't. Spring is coming early yay!\n,0,0


## Compare results

* We notice that before fine tuning model had an accuracy of 13% against the sms spam data set.
* After PEFT fine tuning the model has an accuracy of 98% against the sms spam data set.